# Run EDDY on UKB dataset

In [4]:
import os
import numpy as np
import glob
import subprocess
import sys
import json
import pandas as pd
from pandas.io.json import json_normalize 

rootdir = '/home/preclineu/ramcir/Desktop/Diffusion/'

#submit job to cluster command
cluster_log_dir = '/home/preclineu/ramcir/DCCN/TorqueLogs'
cmd_qsub_base = ['/home/preclineu/ramcir/DCCN/Scripts/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '20gb',
                 '-logfiledir', cluster_log_dir]

#get a list of subjects to process
sub_dirs = sorted(glob.glob(os.path.join(rootdir,'Subjects','*')))
print('Found', len(sub_dirs), 'subjects in total')

Found 3 subjects in total


### Sort subjects

In [5]:
sub_no_dmri = []
sub_no_eddy = []
sub_ready = []
sub_complete = []

# Control the number of subjects
sdirs = sub_dirs[0:]

for s in sdirs[0:]:
    subid = os.path.splitext(os.path.basename(s))[0]

    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    new_eddy = os.path.join(dmridir, 'data2.qc', 'qc.json')  
    eddyfile = os.path.join(dmridir, 'data.eddy_parameters')
    
    try:
        #Check if dMRI data is avalable
        if os.path.exists(dmridir) == False:  
#             print(subid,' dMRI data not found.')
            sub_no_dmri.append(subid)
        else: 
            #Check if EDDY files are available
            if os.path.exists(eddyfile) == False:
#                 print(subid,' EDDY data not found.')
                sub_no_eddy.append(subid)
            else:  
                #Check if quad has already been run
                if os.path.exists(new_eddy) == True:
#                     print(subid,' QC already done.') 
                    sub_complete.append(s)
                else:
#                     print(subid,' is ready for QC') 
                    sub_ready.append(s)
    except:
        print(subid, ' ignored due to', sys.exc_info()[0], '. Continuing with the next subject.')
        
print('Found', len(sub_no_dmri), 'subjects who are missing dMRI data')
print('Found', len(sub_no_eddy), 'subjects who are missing eddy')
print('Found', len(sub_ready), 'subjects ready to process')
print('Found', len(sub_complete), 'subjects already processed')

Found 1 subjects who are missing dMRI data
Found 0 subjects who are missing eddy
Found 2 subjects ready to process
Found 0 subjects already processed


### Run Eddy

##### ! This function requires FSL version 6.0.6

In [ ]:
for s in sub_ready[0:]:  
    #Define directories needed
    subid = os.path.splitext(os.path.basename(s))[0]
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    rawdir = os.path.join(s, 'dMRI', 'raw')

    #Write command for running EDDY      
    eddy_cmd = ['eddy_cpu ',
                 '--init=' + dmridir + '/data.eddy_parameters ',
                 '--imain=' + dmridir + '/data.nii.gz ',
                 '--mask=' + dmridir + '/b0_average_brain_mask.nii.gz ', 
                 '--index=' + dmridir + '/index.txt ',
                 '--acqp=' + dmridir + '/acqparams.txt ',
                 '--bvecs=' + dmridir + '/bvecs ',
                 '--bvals=' + dmridir + '/bvals ',
                 '--out='  + dmridir + '/data2 ',
                 '--niter=0 --cnr_maps'] 
    
    proc_cmd = "%s" % str(' '.join(eddy_cmd)) # Join commands together if you wish to run them at the same time
    cmd_qsub = cmd_qsub_base + ['-name', 'UKB_fslmaths_' + subid ,'-command " ', proc_cmd, '"'] # command for submitting job to cluster
    print(' '.join(cmd_qsub))
    subprocess.run(' '.join(cmd_qsub), shell=True) #Run command      